In [0]:
pip install python_jwt

In [0]:
pip install gcloud


In [0]:
pip install sseclient


In [0]:
pip install pycrypto


In [0]:
pip install requests-toolbelt

In [0]:
pip install firebase

In [0]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical as tcg
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD
import numpy as np
import cv2
from firebase import firebase as fb
from google.cloud import storage as st


In [0]:
(xtr,ytr),(xte,yte) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
xtr = xtr.astype('float32')/32
xte = xte.astype('float32')/32

In [0]:
ytr = tcg(ytr)
yte = tcg(yte)

In [0]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(7,7), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(7,7), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(.5))

model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(.5))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(.5))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(.5))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(.5))

model.add(Conv2D(filters=1024, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=10, kernel_size=(3,3), padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(xtr,ytr, validation_data=(xte,yte), epochs=200, batch_size=4096)
score = model.evaluate(xte,yte)
print("Test score:",score[0])
print("Text Accuracy:",score[1]*100)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 27s 534us/step - loss: 2.2515 - acc: 0.2010 - val_loss: 9.5692 - val_acc: 0.1645
Epoch 2/200
50000/50000 [==============================] - 8s 153us/step - loss: 1.8054 - acc: 0.3180 - val_loss: 9.5773 - val_acc: 0.1811
Epoch 3/200
50000/50000 [==============================] - 8s 152us/step - loss: 1.6727 - acc: 0.3690 - val_loss: 3.1545 - val_acc: 0.2519
Epoch 4/200
50000/50000 [==============================] - 8s 153us/step - loss: 1.5845 - acc: 0.4075 - val_loss: 2.3738 - val_acc: 0.2895
Epoch 5/200
50000/50000 [==============================] - 8s 152us/step - loss: 1.5249 - acc: 0.4316 - val_loss: 1.7736 - val_acc: 0.3991
Epoch 6/200
50000/50000 [====

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
storage_client = st.Client.from_service_account_json('/content/drive/My Drive/JSON/objectdtection-feabc8d45d12.json')
bucket=storage_client.get_bucket('objectdtection.appspot.com')
image=bucket.blob('ddd.jpg')

In [0]:
url=image.public_url
import urllib
req=urllib.request.urlopen(url)
imageF=np.asarray(bytearray(req.read()),dtype=np.uint8)
img=cv2.imdecode(imageF,-1)
requiredSize = (32,32)
imageUpload = cv2.resize(img, requiredSize)
imageUpload = np.reshape(imageUpload,(1,32,32,3))


In [0]:
bestProbability = model.predict_proba(imageUpload)
availableClasses = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
bestProbability = bestProbability.astype('int')
maxValue = np.argmax(bestProbability)
answer = availableClasses[maxValue]

count = 0
for i in bestProbability[0]:
  if bestProbability[0][0] == i:
    count = count + 1
if count == 10:
  print("Not Recognized")
else:
  answerOutput = "The Recognized Object is a : " + answer
  print(answerOutput)
  fire=fb.FirebaseApplication('https://objectdtection.firebaseio.com/')
  fire.put('https://objectdtection.firebaseio.com/','Answer',answerOutput)

The Recognized Object is a : truck
